In [2]:
import pandas as pd

data = pd.read_parquet('modelo_regresion.parquet')


In [3]:
data

,business_id,business_name,city,latitude,longitude,business_rating,review_text,review_stars,review_date,gmap_id
0,aNtKyc2rr-uK5cqzY9TVQQ,Chipotle Mexican Grill,Largo,27.894167,-82.779866,3.0,The New Chipotle Mexican Grill conveniently lo...,4,2017-08-30,None
1,aNtKyc2rr-uK5cqzY9TVQQ,Chipotle Mexican Grill,Largo,27.894167,-82.779866,3.0,I am a chipotle enthusiast but from the first ...,5,2018-01-18,None
2,aNtKyc2rr-uK5cqzY9TVQQ,Chipotle Mexican Grill,Largo,27.894167,-82.779866,3.0,Went into this location about an hour before t...,5,2018-09-08,None
3,aNtKyc2rr-uK5cqzY9TVQQ,Chipotle Mexican Grill,Largo,27.894167,-82.779866,3.0,I come to this chipotle often and it's one of ...,5,2019-06-24,None
4,aNtKyc2rr-uK5cqzY9TVQQ,Chipotle Mexican Grill,Largo,27.894167,-82.779866,3.0,"The food is good, as per usual at any Chipotle...",2,2018-06-20,None
...,...,...,...,...,...,...,...,...,...,...
61892,None,Taco Bus Miami,Miami,25.760594,-80.409314,4.1,None,1,2020-04-01,0x88d9bf290ee132e7:0x771f3f6950e264db
61893,None,Taco Bus Miami,Miami,25.760594,-80.409314,4.1,None,5,2021-03-31,0x88d9bf290ee132e7:0x771f3f6950e264db
61894,None,Taco Bus Miami,Miami,25.760594,-80.409314,4.1,None,5,2020-09-14,0x88d9bf290ee132e7:0x771f3f6950e264db
61895,None,Taco Bus Miami,Miami,25.760594,-80.409314,4.1,None,5,2020-09-28,0x88d9bf290ee132e7:0x771f3f6950e264db


## REGRESION LINEAL CON RANDOM FOREST 
*Este modelo puede ser usado para predecir el business_rating de un restaurante Taco Bell basado en la fecha de la reseña y la ciudad. Podría ser útil para anticipar cómo podría ser la calificación en los próximos años y así planificar estrategias para mejorar la calidad del servicio o tomar decisiones de expansión.*

In [197]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import OneHotEncoder

# Cargar los datos desde el archivo parquet
data = pd.read_parquet('modelo_regresion.parquet')

# Convertir la columna review_date a formato numérico
data['review_date'] = pd.to_datetime(data['review_date']).dt.strftime('%Y%m%d').astype(int)

# Filtrar los datos para los restaurantes Taco Bell
taco_bell_data = data[data['business_name'].str.contains('Taco Bell', case=False)]

# Seleccionar las características y la variable objetivo
features = ['review_date', 'city']
target = 'business_rating'

# Codificar la columna 'city' utilizando one-hot encoding
encoder = OneHotEncoder(handle_unknown='ignore')
city_encoded = encoder.fit_transform(taco_bell_data[['city']])

# Convertir el resultado de one-hot encoding a un DataFrame
city_encoded_df = pd.DataFrame(city_encoded.toarray(), columns=encoder.get_feature_names_out(['city']))

# Concatenar las características codificadas con las otras características
taco_bell_data_encoded = pd.concat([taco_bell_data[['review_date']].reset_index(drop=True), city_encoded_df], axis=1)

# Dividir los datos en conjunto de entrenamiento y conjunto de prueba
X_train, X_test, y_train, y_test = train_test_split(taco_bell_data_encoded, taco_bell_data[target], test_size=0.2, random_state=42)

# Crear el modelo de regresión de bosque aleatorio
model = RandomForestRegressor(random_state=42)
model.fit(X_train, y_train)

# Realizar predicciones en el conjunto de prueba
y_pred = model.predict(X_test)

# Calcular el error cuadrático medio
mse = mean_squared_error(y_test, y_pred)
print("Mean Squared Error:", mse)

Mean Squared Error: 0.18875514185720832


In [198]:
# Ejemplo de uso: predecir el business_rating para Taco Bell en los próximos años
new_data = pd.DataFrame({'review_date': [20230101, 20240101, 20250101], 'city': ['Miami', 'Orlando', 'Tampa']})

# Codificar la columna 'city' del nuevo conjunto de datos utilizando el mismo encoder
new_city_encoded = encoder.transform(new_data[['city']])

# Convertir el resultado de one-hot encoding a un DataFrame
new_city_encoded_df = pd.DataFrame(new_city_encoded.toarray(), columns=encoder.get_feature_names_out(['city']))

# Concatenar las características codificadas con la columna 'review_date'
new_data_encoded = pd.concat([new_data[['review_date']], new_city_encoded_df], axis=1)

# Asegurarse de que las columnas del nuevo conjunto de datos coincidan con las del conjunto de entrenamiento
missing_cols = set(X_train.columns) - set(new_data_encoded.columns)
for c in missing_cols:
    new_data_encoded[c] = 0
new_data_encoded = new_data_encoded[X_train.columns]

# Realizar predicciones con el modelo
new_predictions = model.predict(new_data_encoded)

print("Predicted business_rating for Taco Bell in the next years:")
for year, prediction in zip([2023, 2024, 2025], new_predictions):
    print(f"Year {year}: {prediction}")


Predicted business_rating for Taco Bell in the next years:
Year 2023: 2.21
Year 2024: 2.21
Year 2025: 1.71


In [155]:
#mostrar todos los business_name que tengan taco bell en su nombre y que pertenezcan a la city Tampa
data[(data['business_name'].str.contains('Taco Bell')) & (data['city'] == 'Tampa')]

,business_id,business_name,city,latitude,longitude,business_rating,review_text,review_stars,review_date,gmap_id
232,H2nFgEFitRk7ltE8IRH0xA,Taco Bell,Tampa,27.944918,-82.523213,2.5,This Taco Bell lives up to the standards that ...,5,2017-05-26,None
233,H2nFgEFitRk7ltE8IRH0xA,Taco Bell,Tampa,27.944918,-82.523213,2.5,"It's Taco Bell. It's crap food, terrible servi...",1,2018-01-01,None
234,H2nFgEFitRk7ltE8IRH0xA,Taco Bell,Tampa,27.944918,-82.523213,2.5,"Was staying at hotel across street, got in lat...",1,2018-05-13,None
235,H2nFgEFitRk7ltE8IRH0xA,Taco Bell,Tampa,27.944918,-82.523213,2.5,Now I see why this location has such terrible ...,1,2018-07-03,None
236,H2nFgEFitRk7ltE8IRH0xA,Taco Bell,Tampa,27.944918,-82.523213,2.5,"Waited 45 minutes at the drive thru, surely em...",1,2017-12-27,None
...,...,...,...,...,...,...,...,...,...,...
42533,3Vax2SswY9_5KSZoCJLyIA,Taco Bell,Tampa,28.025578,-82.561893,2.0,Every. Single. Time. The food is so sloppy tha...,1,2020-01-09,None
42534,3Vax2SswY9_5KSZoCJLyIA,Taco Bell,Tampa,28.025578,-82.561893,2.0,maybe it's because i'm pregnant but i'm obsess...,5,2020-06-11,None
42535,3Vax2SswY9_5KSZoCJLyIA,Taco Bell,Tampa,28.025578,-82.561893,2.0,Awful.. the blonde at the drive through is obn...,1,2017-12-13,None
42536,3Vax2SswY9_5KSZoCJLyIA,Taco Bell,Tampa,28.025578,-82.561893,2.0,I made a huge mistake stopping here. Taco Bel...,1,2021-12-29,None


In [180]:
#el promedio de business_rating para todos los business_name que sean Taco Bell
data[data['business_name'].str.contains('Taco Bell')]['business_rating'].mean()


2.799644830307814

In [181]:
#el promedio de business_rating para todos los business_name que no sean Taco Bell
data[~data['business_name'].str.contains('Taco Bell')]['business_rating'].mean()

#La competencia esta teniendo mejor rating que Taco Bell


3.942029547024241

In [60]:
#tipo de dato en cada columna
data.dtypes


business_id         object
business_name       object
city                object
latitude           float64
longitude          float64
business_rating    float64
review_text         object
review_stars         int64
review_date         object
gmap_id             object
dtype: object

## REGRESION LINEAL CON RANDOM FOREST (adiocional con latitud y longitud)
*El propósito del modelo es predecir la calificación (business_rating) de un restaurante en función de la fecha de la reseña, la latitud y la longitud. Esto puede ayudar a identificar tendencias y factores que influyen en las calificaciones, permitiendo a los dueños de restaurantes tomar medidas proactivas para mejorar las calificaciones.*

In [116]:
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error


In [123]:
# Cargar los datos desde el archivo parquet
data = pd.read_parquet('modelo_regresion.parquet')

# Convertir la columna review_date a formato numérico
data['review_date'] = pd.to_datetime(data['review_date']).dt.strftime('%Y%m%d').astype(int)

# Filtrar los datos solo para los locales de Taco Bell
taco_bell_data = data[data['business_name'] == 'Taco Bell']

In [124]:
# Seleccionar las características y el target
features = ['review_date']
target = 'business_rating'
X = taco_bell_data[features]
y = taco_bell_data[target]

# Dividir los datos en conjunto de entrenamiento y conjunto de prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Crear el modelo de Random Forest Regressor
model = RandomForestRegressor(random_state=42)

# Entrenar el modelo
model.fit(X_train, y_train)

# Realizar predicciones en el conjunto de prueba
y_pred = model.predict(X_test)

# Calcular el error cuadrático medio
mse = mean_squared_error(y_test, y_pred)
print("Mean Squared Error:", mse)

Mean Squared Error: 0.7369314128110381


In [127]:
# Ejemplo de uso: predecir el business_rating para Taco Bell en los próximos años
new_data = pd.DataFrame({'review_date': [20220101, 20250102, 20250103]})
new_predictions = model.predict(new_data)
print("Predicted business_rating for Taco Bell in the future:", new_predictions)

Predicted business_rating for Taco Bell in the future: [2.18958333 2.18958333 2.18958333]


## SENTIMENT ANALISIS + Modelo de clasificacion 
*El propósito del modelo es predecir el sentimiento (positivo, negativo o neutral) de una nueva reseña basada en su contenido textual mediante un modelo de clasificacion. Esto puede ayudar a un restaurante a monitorear y responder proactivamente a las opiniones de los clientes.*

In [6]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report

# Cargar los datos desde el archivo parquet
data = pd.read_parquet('modelo_regresion.parquet')

# Reemplazar los valores None o NaN con una cadena vacía
data['review_text'] = data['review_text'].fillna('')

# Convertir el texto a minúsculas
data['review_text'] = data['review_text'].str.lower()

# Realizar análisis de sentimiento utilizando TextBlob
from textblob import TextBlob
data['sentiment'] = data['review_text'].apply(lambda x: TextBlob(x).sentiment.polarity)

# Asignar etiquetas de sentimiento
data['sentiment'] = data['sentiment'].apply(lambda x: 'positive' if x > 0 else 'negative' if x < 0 else 'neutral')

# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(data['review_text'], data['sentiment'], test_size=0.2, random_state=42)

# Crear el vectorizador TF-IDF para convertir el texto en vectores numéricos para el modelo
vectorizer = TfidfVectorizer()
X_train_vectorized = vectorizer.fit_transform(X_train)
X_test_vectorized = vectorizer.transform(X_test)

# Crear el modelo de clasificación Naive Bayes Multinomial para análisis
model = MultinomialNB()

# Entrenar el modelo de clasificación
model.fit(X_train_vectorized, y_train)

# Realizar predicciones en el conjunto de prueba
y_pred = model.predict(X_test_vectorized)

# Generar el informe de clasificación
classification_report = classification_report(y_test, y_pred)
print(classification_report)

              precision    recall  f1-score   support

    negative       0.92      0.04      0.07      1599
     neutral       0.70      0.01      0.01      1348
    positive       0.77      1.00      0.87      9433

    accuracy                           0.77     12380
   macro avg       0.79      0.35      0.32     12380
weighted avg       0.78      0.77      0.67     12380



In [11]:
# Use the trained model to make predictions on new data
new_data = pd.DataFrame({'review_text': ['This restaurant is amazing!','terrible' ]})

# Preprocess the new data
new_data['review_text'] = new_data['review_text'].fillna('')
new_data['review_text'] = new_data['review_text'].str.lower()

# Vectorize the new data
new_data_vectorized = vectorizer.transform(new_data['review_text'])

# Make predictions using the trained model
new_predictions = model.predict(new_data_vectorized)

# Print the predictions
print(new_predictions)

['positive' 'negative']


In [194]:
#muestra 10 ejemplos de reviews con sentimiento negativo
data[data['sentiment'] == 'negative']['review_text'].head(10)


6      the kids working here may still be "in trainin...
10     i always get disappointed when i go to chipotl...
14     this was a terrible experience i had ordered 2...
15     not great at all! cashier touched my food whil...
17     this chipotle location is the worst!  it's the...
77     for $8.30 (including $1 tip) i had two very ti...
128    i hate to be outnumbered here  but this place ...
146    in one sentence: a taco truck that's now in a ...
173    i have been living close to this taco joint fo...
183    the atmosphere here is neat, but i was put off...
Name: review_text, dtype: object

### RESULTADOS DE SENTIMENT ANALYSIS CON TEXTBLOB

In [191]:
# para el business_name 'Taco Bell', contar cuántas reseñas tienen sentimiento positivo, negativo y neutral y calcualar el porcentaje de cada uno
taco_bell_data = data[data['business_name'] == 'Taco Bell']
sentiment_counts = taco_bell_data['sentiment'].value_counts()
sentiment_counts

sentiment
positive    887
neutral     854
negative    793
Name: count, dtype: int64

In [192]:
sentiment_percentages = sentiment_counts / sentiment_counts.sum() * 100
print(sentiment_percentages)

sentiment
positive    35.003946
neutral     33.701657
negative    31.294396
Name: count, dtype: float64


### Regresion Logistica (NO VA)
*En base a reviews analizadas sobre restaurantes mexicanos en Florida, detecta cuales son las palabras clave que mas se relacionan con un buen business_rating (4-5). Elijo la puntuacion que quiero tener y veo atributos*. A medida que se va alimentando de mas informacion puede que otras palabras tomen mas relevancia a la hora de relacionarse con una buena puntuacion 

In [183]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

# Cargar los datos desde el archivo parquet
data = pd.read_parquet('modelo_regresion.parquet')

# Reemplazar los valores None o NaN con una cadena vacía
data['review_text'] = data['review_text'].fillna('')

# Convertir el texto a minúsculas
data['review_text'] = data['review_text'].str.lower()

# Filtrar los datos para obtener solo las reseñas con rating_stars de 4 o 5 
positive_reviews = data[data['review_stars'].isin([4, 5])]

# Dividir los datos en conjuntos de entrenamiento y prueba (80% de entrenamiento, 20% de prueba)
X_train, X_test, y_train, y_test = train_test_split(positive_reviews['review_text'], positive_reviews['review_stars'], test_size=0.2, random_state=42)

# Crear el vectorizador TF-IDF y transformar los datos de entrenamiento y prueba
vectorizer = TfidfVectorizer()
X_train_vectorized = vectorizer.fit_transform(X_train)
X_test_vectorized = vectorizer.transform(X_test)

# Crear el modelo de Regresión Logística y entrenarlo con los datos de entrenamiento vectorizados 
model = LogisticRegression()

# Entrenar el modelo de Regresión Logística con los datos de entrenamiento vectorizados
model.fit(X_train_vectorized, y_train)

# Obtener los coeficientes de las características y los nombres de las características 
feature_names = vectorizer.get_feature_names_out()
coefficients = model.coef_[0]

# Obtener las 10 principales palabras relacionadas con una buena puntuación 
top_words_indices = coefficients.argsort()[-10:][::-1]
top_words = [feature_names[i] for i in top_words_indices]

# Imprimir las 5 principales palabras relacionadas con una buena puntuación
print("Las 10principales palabras relacionadas con una buena puntuación son:")
for word in top_words:
    print(word)

Las 10principales palabras relacionadas con una buena puntuación son:
amazing
best
incredible
everything
awesome
delicious
wow
hands
fantastic
perfect


## Regresion lineal (Puede ir)
*Predice el potencial puntaje podria tener un local tomando de referencia la ubicacion donde lo ponga*


In [136]:
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import pandas as pd

# Cargar los datos desde el archivo parquet
data = pd.read_parquet('modelo_regresion.parquet')

# Convertir la columna review_date a formato numérico
data['review_date'] = pd.to_datetime(data['review_date']).dt.strftime('%Y%m%d').astype(int)

# Seleccionar las características y la variable objetivo
features = ['latitude', 'longitude']
target = 'business_rating'

# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(data[features], data[target], test_size=0.2, random_state=42)

# Crear el modelo de Regresión Lineal
model = LinearRegression()

# Entrenar el modelo
model.fit(X_train, y_train)

# Realizar predicciones en el conjunto de prueba
y_pred = model.predict(X_test)

# Calcular el error cuadrático medio
mse = mean_squared_error(y_test, y_pred)
print('Mean Squared Error:', mse)


Mean Squared Error: 0.391858624832528


In [138]:
# Assuming you have already trained the model and have the necessary data

# Create a new dataframe with the features for prediction
new_data = pd.DataFrame({'latitude': [27.961018, 27.979929, 27.938770, 28.646935, 26.470094],
                         'longitude': [-82.463937, -82.826317, -82.482742, -81.270090, -81.792269]})

# Make predictions on the new data
predictions = model.predict(new_data)

# Print the predicted business ratings
for i, prediction in enumerate(predictions):
    print(f"Predicted business rating for New Taco Bell {i+1}: {prediction}")

Predicted business rating for New Taco Bell 1: 3.8922955193744437
Predicted business rating for New Taco Bell 2: 3.8645013356635713
Predicted business rating for New Taco Bell 3: 3.892213087404537
Predicted business rating for New Taco Bell 4: 3.940060174300947
Predicted business rating for New Taco Bell 5: 4.0290414895546105
